# MNIST 模型強化

## 步驟1：載入 MNIST 手寫阿拉伯數字資料

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

# 載入 MNIST 手寫阿拉伯數字資料
(x_train, y_train),(x_test, y_test) = mnist.load_data()


## 步驟2：資料清理，此步驟無需進行

## 步驟3：進行特徵工程，將特徵縮放成(0, 1)之間

# 特徵縮放，使用常態化(Normalization)，公式 = (x - min) / (max - min)
# 顏色範圍：0~255，所以，公式簡化為 x / 255
# 注意，顏色0為白色，與RGB顏色不同，(0,0,0) 為黑色。
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

In [2]:
x_train_norm.shape

(60000, 28, 28)

## 步驟2：改用 CNN 模型

In [3]:
# 建立模型
from tensorflow.keras import layers
import numpy as np

input_shape=(28, 28, 1)
# 增加一維在最後面
x_train_norm = np.expand_dims(x_train_norm, -1)
x_test_norm = np.expand_dims(x_test_norm, -1)

# CNN 模型
# model = tf.keras.Sequential(
#     [
#         tf.keras.Input(shape=input_shape),
#         layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Flatten(),
#         layers.Dropout(0.5),
#         layers.Dense(10, activation="softmax"),
#     ]
# )

In [4]:
from tensorflow.keras.layers import *

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=input_shape))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(10, activation='softmax'))


In [16]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1)


In [19]:
# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 模型訓練
# history = model.fit(x_train_norm, y_train, epochs=15, validation_split=0.2)
batch_size=64
history = model.fit(datagen.flow(x_train_norm, y_train), epochs=15,
          validation_data=datagen.flow(x_test_norm, y_test, batch_size=batch_size), verbose=2)


# 評分(Score Model)
score=model.evaluate(x_test_norm, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model.metrics_names[i]}: {score[i]:.4f}')

Epoch 1/15
1875/1875 - 18s - loss: 0.1170 - accuracy: 0.9676 - val_loss: 0.0466 - val_accuracy: 0.9858 - 18s/epoch - 10ms/step
Epoch 2/15
1875/1875 - 17s - loss: 0.0772 - accuracy: 0.9776 - val_loss: 0.0392 - val_accuracy: 0.9877 - 17s/epoch - 9ms/step
Epoch 3/15
1875/1875 - 18s - loss: 0.0706 - accuracy: 0.9801 - val_loss: 0.0341 - val_accuracy: 0.9882 - 18s/epoch - 10ms/step
Epoch 4/15
1875/1875 - 17s - loss: 0.0661 - accuracy: 0.9812 - val_loss: 0.0342 - val_accuracy: 0.9888 - 17s/epoch - 9ms/step
Epoch 5/15
1875/1875 - 16s - loss: 0.0629 - accuracy: 0.9822 - val_loss: 0.0312 - val_accuracy: 0.9900 - 16s/epoch - 9ms/step
Epoch 6/15
1875/1875 - 18s - loss: 0.0627 - accuracy: 0.9832 - val_loss: 0.0358 - val_accuracy: 0.9904 - 18s/epoch - 9ms/step
Epoch 7/15
1875/1875 - 19s - loss: 0.0593 - accuracy: 0.9835 - val_loss: 0.0297 - val_accuracy: 0.9912 - 19s/epoch - 10ms/step
Epoch 8/15
1875/1875 - 18s - loss: 0.0592 - accuracy: 0.9829 - val_loss: 0.0330 - val_accuracy: 0.9908 - 18s/epoch 

In [21]:
# 讀取影像並轉為單色
from skimage import io
from skimage.transform import resize
import numpy as np

for i in range(10):
    uploaded_file = f'../extra/myDigits/{i}.png'
    image1 = io.imread(uploaded_file, as_gray=True)

    # 縮為 (28, 28) 大小的影像
    image_resized = resize(image1, (28, 28), anti_aliasing=True)    
    X1 = image_resized.reshape(1,28, 28, 1) #/ 255

    # 反轉顏色，顏色0為白色，與 RGB 色碼不同，它的 0 為黑色
    X1 = np.abs(1-X1)

    # 預測
    #predictions = model.predict_classes(X1)
    predictions = np.argmax(model.predict(X1), axis=-1)
    print(predictions)

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]


In [22]:
# 模型存檔
model.save('cnn_da_model.h5')